# Batch anomaly detector POC

The Anomaly Detector API's batch detection endpoint allows to detect anomalies through the entire times series data. In this detection mode, a single statistical model is created and applied to each point in the data set. If time series has the below characteristics, it is recommended to use batch detection to preview your data in one API call.

* A seasonal time series, with occasional anomalies.
* A flat trend time series, with occasional spikes/dips.

Reference - https://docs.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/concepts/anomaly-detection-best-practices

To send requests to the Anomaly Detector API, we require anomaly detector access key and enpoint for the anomaly detector resource. These can be found in `Keys and Endpoint` menu item on left navigation bar.

In [ ]:
apikey = '<Your api key>'
endpoint = '<Your endpoint>'
batch_detection_endpoint = f'{endpoint}/anomalydetector/v1.1-preview/timeseries/entire/detect'

# Steps to run

* Verify `apikey` and `endpoint` variables above have been updated with correct values.
* Click on `Kernel` menu.
* Click on `Restart & Run All` menu item.

In [ ]:
import requests
import json

**Below is the function that'll make calls to Anomaly Detector API's batch detection endpoint**

In [ ]:
def detect(endpoint, apikey, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': apikey}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

**Below is the function that gets sample data from microsoft's github repo**

In [ ]:
def get_sample_data():
    url = 'http://raw.githubusercontent.com/Azure-Samples/AnomalyDetector/master/ipython-notebook/sample.json'
    response = requests.get(url, verify=False)
    return response.text

## Result
**Below is the code that fetches sample data and calls anomaly detector endpoint.**

Anomaly detector endpoint returns anomaly information for each point in the sample data. Final output printed by this code is combined anomaly output.

In [ ]:
# Ignore the certifcate warning in the final output.

data = get_sample_data()
json_data = json.loads(data)
json_data['sensitivity'] = 95

result = detect(batch_detection_endpoint, apikey, json_data)
combined_result = {
    'granularity': json_data['granularity'],
    'data': []
}

for index, point in enumerate(json_data['series']):
    combined_result['data'].append({
        'timestamp': point['timestamp'],
        'value': point['value'],
        'is_anomaly': result['isAnomaly'][index],
        'is_negative_anomaly': result['isNegativeAnomaly'][index],
        'is_positive_anomaly': result['isPositiveAnomaly'][index]
    })

print(json.dumps(combined_result, indent=2))